Transformação e normalização
============================



## DataFrame



Vamos iniciar criando um `DataFrame`.



In [1]:
import pandas as pd

dicionario_contendo_os_dados = {
    "Nome": ["Platão", "Sócrates", "Descartes", "Nietzsche"],
    "Idade": [80, 71, 53, 55.0],
    "Bigode épico?": [True, True, False, True],
    "Altura": [pd.NA, pd.NA, 1.55, 1.70],
}

df = pd.DataFrame.from_dict(dicionario_contendo_os_dados)

print(df)

        Nome  Idade  Bigode épico? Altura
0     Platão   80.0           True   <NA>
1   Sócrates   71.0           True   <NA>
2  Descartes   53.0          False   1.55
3  Nietzsche   55.0           True    1.7


## Transformações



Parte do tratamento de dados envolve aplicarmos funções nos dados. Chamamos isso de transformação. Quem sabe você tenha uma temperatura em graus Celsius e queira ela em Kelvin. Neste caso, você terá que aplicar uma função que altere todos os valores da coluna original adicionando 273,15 a eles.

Vamos converter a idade dos nossos filósofos de anos para segundos! Note que estamos fazendo cópias do `DataFrame` original para cada alteração. Isso não é obrigatório nem necessário. Fizemos isso aqui para não ter problema na ordem de execução das células deste notebook. Inclusive, se seu conjunto de dados for muito grande, pode não ser uma boa prática ficar criando cópias!



In [2]:
df2 = df.copy()
df2["Idade"] = df2["Idade"] * 356 * 24 * 3600

print(df2)

        Nome         Idade  Bigode épico? Altura
0     Platão  2.460672e+09           True   <NA>
1   Sócrates  2.183846e+09           True   <NA>
2  Descartes  1.630195e+09          False   1.55
3  Nietzsche  1.691712e+09           True    1.7


Agora temos um problema, a coluna `Idade` contém valores muito grandes! Quem sabe seja razoável aplicar um logaritmo aqui!



In [3]:
import numpy as np

df2["Idade"] = np.log10(df2["Idade"])
print(df2)

        Nome     Idade  Bigode épico? Altura
0     Platão  9.391054           True   <NA>
1   Sócrates  9.339222           True   <NA>
2  Descartes  9.212240          False   1.55
3  Nietzsche  9.228326           True    1.7


Faz parte do pré-processamento de dados refletir sobre quais transformações são interessantes/necessárias nos seus dados. Em particular, <u>se você tem alguma grandeza numérica que varia mais que uma ordem de grandeza</u>, considere a possibilidade de trabalhar com o logaritmo dessa grandeza para evitar que os números muito grandes dominem os cálculos em relação aos números muito pequenos.



## Normalização



Um tipo de transformação de dados muito comum é a chamada *normalização*. Muitos algoritmos de aprendizado de máquina <u>requerem</u> que os dados estejam normalizados. Redes neurais artificiais, por exemplo, se beneficiam quando os dados de entrada estão entre $-1$ e $1$. Certos modelos assumem que os dados vêm de uma distribuição com média zero e desvio padrão um e irão se comportar de maneira subótima quando esse não é o caso.

Existem três tipos usuais de normalizações lineares para dados numéricos tabulados: normalização padrão (z-score), normalização pelo máximo absoluto e normalização pelo mínimo e máximo.



### Normalização padrão



A <u>normalização padrão</u> altera a média dos dados para zero e o desvio padrão dos dados para um. Ela pode ser calculada com a equação abaixo, onde $\mu$ é a média dos dados $x$, $\sigma$ é o desvio padrão dos dados $x$, $x_i$ é o exemplo $i$ do conjunto de dados $x$ e $z_i$ é o valor normalizado do exemplo $i$ do conjunto de dados $x$.

$$z_i = \frac{x_i - \mu}{\sigma}$$



In [4]:
media = df['Idade'].mean()
desvio_padrao = df['Idade'].std()

df['Idade_zscore'] = (df['Idade'] - media) / desvio_padrao

print(df)

        Nome  Idade  Bigode épico? Altura  Idade_zscore
0     Platão   80.0           True   <NA>      1.175689
1   Sócrates   71.0           True   <NA>      0.481840
2  Descartes   53.0          False   1.55     -0.905858
3  Nietzsche   55.0           True    1.7     -0.751670


Nesta normalização, nosso interesse *não* é inferir a média e o desvio padrão da população. Desta forma, computamos $\mu$ e $\sigma$ usando as formulas para populações e não para amostras (os próprios símbolos já sugerem isso também).

Note que esta transformação não altera o quão normalmente distribuídos estão seus dados, apenas translada e &ldquo;estica/comprime&rdquo; seus dados para que apresentem média 0 e desvio padrão 1.

Certos algoritmos esperam que os dados tenham distribuições comparáveis. Um exemplo são algoritmos baseados em distâncias como o $k$​-vizinhos mais próximos. Neste caso, usar o normalizador padrão é uma boa estratégia.



### Normalização pelo máximo absoluto



A <u>normalização pelo máximo absoluto</u> faz com que todos os dados fiquem entre o intervalo $[-1, 1]$. Para calcular esta normalização basta dividir os dados pelo valor máximo absoluto. Veja a equação abaixo que computa o valor transformado $n_i$ do exemplo $i$.

$$ n_i = \frac{x_i}{\max(|x|)} $$



In [5]:
maximo_absoluto = max(abs(df['Idade']))

df['Idade_norm_max_abs'] = df['Idade'] / maximo_absoluto

print(df)

        Nome  Idade  Bigode épico? Altura  Idade_zscore  Idade_norm_max_abs
0     Platão   80.0           True   <NA>      1.175689              1.0000
1   Sócrates   71.0           True   <NA>      0.481840              0.8875
2  Descartes   53.0          False   1.55     -0.905858              0.6625
3  Nietzsche   55.0           True    1.7     -0.751670              0.6875


Uma vantagem da normalização pelo máximo absoluto é que ela mantém a esparsidade dos seus dados (isto é: não altera os valores que são zero). Aliado com a certeza que seus dados estão contidos em um intervalo entre $[-1,1]$ faz deste normalizador uma excelente escolha para treinar modelos como redes neurais artificiais, por exemplo.



### Normalização pelo mínimo e máximo



A <u>normalização pelo mínimo e máximo</u> modifica os dados de forma que, após a transformação, os dados tenham valor mínimo de 0 e valor máximo de 1. O valor transformado $m_i$ do exemplo $i$ depende do valor do atributo $x_i$ deste exemplo e do valor mínimo e valor máximo deste atributo considerando todo o dataset $x$:

$$ m_i = \frac{x_i - \min(x_i)}{\max(x) - \min(x)} $$



In [6]:
maximo = df['Idade'].max()
minimo = df['Idade'].min()

df['Idade_norm_min_max'] = (df['Idade'] - minimo) / (maximo - minimo)

print(df)

        Nome  Idade  Bigode épico? Altura  Idade_zscore  Idade_norm_max_abs   
0     Platão   80.0           True   <NA>      1.175689              1.0000  \
1   Sócrates   71.0           True   <NA>      0.481840              0.8875   
2  Descartes   53.0          False   1.55     -0.905858              0.6625   
3  Nietzsche   55.0           True    1.7     -0.751670              0.6875   

   Idade_norm_min_max  
0            1.000000  
1            0.666667  
2            0.000000  
3            0.074074  


Este normalizador tem a vantagem que você sabe exatamente o valor máximo e mínimo dos dados normalizados, facilitando a interpretação dos seus resultados. Note que você pode perder a esparsidade dos dados quando aplicar este normalizador em situações onde existem números positivos e negativos. Costuma não ser uma boa estratégia perder a esparsidade dos dados!



## Normalização utilizando o `scikit-learn`



A normalização de dados em experimentos de Aprendizado de Máquina realizados em Python é geralmente feita pelo excelente módulo `scikit-learn`. Existe uma seção inteira sobre isso no [Guia do Usuário](https://scikit-learn.org/stable/modules/preprocessing.html), vale muito a pena dar uma lida! Os normalizadores estão todos dentro do submódulo `preprocessing`. Veja na tabela abaixo.

| Nome dentro do <code>scikit-learn</code>|Tipo de normalizador|
|---|---|
| <code>StandardScaler</code>|Normalizador padrão|
| <code>MaxAbsScaler</code>|Normalizador pelo máximo absoluto|
| <code>MinMaxScaler</code>|Normalizador pelo mínimo e máximo|

Todos os normalizadores do `scikit-learn` funcionam da mesma maneira:

1.  Garantir que os dados têm duas dimensões (requerimento do `scikit-learn`, não é uma limitação da técnica de normalização em si)
2.  Criamos uma instância do normalizador
3.  Ajustamos o normalizador aos dados
4.  Usamos o método `transform` sempre que quisermos aplicar o normalizador
5.  (Opcional) Usamos o método `inverse_transform` sempre que quisermos &ldquo;desnormalizar&rdquo;, ou seja, retornar dados normalizados para os valores não-normalizados.

Vamos ver um exemplo com o `MinMaxScaler`.



In [7]:
from sklearn.preprocessing import MinMaxScaler

# Passo 1: só é necessário se os dados tem apenas uma dimensão
# Se seus dados já tem duas dimensões, não precisa fazer isso!
x = df["Idade"].values.reshape(-1, 1)

# Passo 2: criar uma instância do normalizador
normalizador = MinMaxScaler()

# Passo 3: ajustar o normalizador aos dados
normalizador.fit(x)

# Passo 4: aplicar o normalizador usando o método `transform`
idade_normalizada = normalizador.transform(x)

# Passo 5: se desejamos recuperar os valores iniciais, usamos o `inverse_transform`
idade_desnormalizada = normalizador.inverse_transform(idade_normalizada)

# Ver resultados
print("Dados originais")
print(x)
print()

print("Dados normalizados:")
print(idade_normalizada)
print()

print("Dados desnormalizados:")
print(idade_desnormalizada)

Dados originais
[[80.]
 [71.]
 [53.]
 [55.]]

Dados normalizados:
[[1.        ]
 [0.66666667]
 [0.        ]
 [0.07407407]]

Dados desnormalizados:
[[80.]
 [71.]
 [53.]
 [55.]]


A estratégia é similar para usar o `StandardScaler` e o `MaxAbsScaler` (ou qualquer outro normalizador do `scikit-learn`) e fica como exercício para o leitor atento. Os normalizadores do `scikit-learn` podem ser treinados em dados com múltiplas colunas ao mesmo tempo, teste para ver como é!

